<a href="https://colab.research.google.com/github/ChaejinE/HanComTermProject/blob/main/Puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from queue import PriorityQueue
from math import sqrt

In [2]:
class State(object):
  def __init__(self, value, parent,
               start = "801234567", goal = "012345678"):
    self.children = []
    self.parent = parent
    self.value = value # "801234567" position을 의미
    self.dist = 0 # cost 를 의미

    if parent: # parent 가 존재한다면
      self.path = parent.path[:] # parent 의 path 를 물려받아 새롭게 추가한다
      self.path.append(value)
      self.start = parent.start # parent 의 start 를 따라간다 (start)
      self.goal = parent.goal # parent 의 goal 을 따라간다 (goal)

    else:
      self.path = [value]
      self.start = start
      self.goal = goal

  def GetDist(self): # 뒤에서 정의
    pass

  def CreateChildren(self): # 뒤에서 정의
    pass

# cost를 계산, 경로를 찾는 코드
class State_Puzzle(State):
  def __init__(self, value, parent, start = 0, goal = 0):
    # 상속받은 State 객체를 불러옴(State 객체의 __init__함수를 실행)
    super(State_Puzzle, self).__init__(value, parent, start, goal)
    self.dist = self.GetDist() # State_Puzzle을 실행함과 동시에 GetDist()를 실행하겠다.

  def GetDist(self):
    # goal 에 도달한 경우
    if self.value == self.goal:
      return 0
    dist = 0 # 시작값

    # 9줄 짜리 goal
    # -> 3*3을 의미함
    # 한줄에 3개가 있따.
    # 3개 -> size
    # self.goal = '012345678'

    # self.goal = '876543210'
    # len(self.goal) = 9

    # 801
    # 345
    # 672

    # 012
    # 345
    # 678

    size = sqrt(len(self.goal)).real # 한줄에 몇 개가 있는가, 실수 값으로 표현

    # 휴리스틱 계산(Manhattan distance 로 계산)
    for n in range(len(self.goal)):
      piece = self.goal[n]
      # 목표 상태(goal state)를 2D로 표현하면 그 위치는 어떻게 되는가
      # goal state = "012345678"
      goal_x = int(n / size) # 1 (4를 기준으로)
      goal_y = n - goal_x * size # 1 (나머지와 같은 의미)
      # 줄글로 표현되어 있는 퍼즐을 실제로 2D space 로 표현했을 때 x, y 좌표가 무엇인가

      # 현재 상태(goal state)를 2D로 표현하면 그 위치는 어떻게 되는가
      # self.value "801234567"
      value_x = int(self.value.index(piece) / size)
      value_y = self.value.index(piece) - value_x * size

      dist += abs(goal_x - value_x) + abs(goal_y - value_y) # hueristic. 앞으로 goal까지 몇칸을 움직어야 하는지 알 수 없
      # 현재 6 이라는 piece가 (2,1) 위치에 있고,
      # 사실 목표로 하는 state 에서, 6은 (2,0)위치에 있어야 함
      # -> "6"이라는 piece에 대한 cost를, |2-2| + |1-0| = 1 이라고 heuristic하게 정의한다. (맨하탄 디스턴스)

    # 012
    # 345
    # 678

    # dist = 1 + 1 + 2 + 0 + 0 + 0 + 0 + 0 + 4 -> 라고 hueristic 을 정의할 것
    # dist = h
    # len(self.path) = g # 현재 상태로 오기 위해서 실제로 몇 번 움직였나
    # f = h+g 로 놓고, f를 최소화 하는 방향으로 A*를 돌릴겁니다
    return dist + len(self.path)

  def CreateChildren(self):

    size = int(sqrt(len(self.goal)).real)
    if not self.children:
      #self.value = "823407561"
      i = self.value.index("0") # (빈칸)
      # 현재 상태에서, 0의 위치를 확인
      # 0의 상하좌우를 봐서 움직일 수 있는 state가 있는지 확인합니다.
      if not int(i % size) == size - 1: # 0의 위치가 2D space 에서 제일 오른쪽줄이 아니라면 (2/5/8이 아니라면)
        val = self.value # "823407561"
        val = val[:i] + val[i+1] + val[i] + val[i+2:] # "8234" + "7" + "0" + "561" => "823470561"
        child = State_Puzzle(val, self)
        self.children.append(child)

      if not int(i % size) == 0: # 0의 위치가 2D space에서 제일 왼쪽줄이 아니라면 (0/3/6이 아니라면)
        val = self.value
        val = val[:i-1] + val[i] + val[i-1] + val[i+1:] # "823" + "0" + "4" + "7561" => "823047561"
        child = State_Puzzle(val, self)
        self.children.append(child)

      if i < len(self.value) - size: # 0의 위치가 맨 아랫줄에 있지 않다면 (0이 위치하는 index 가 6/7/8이 아니라면)
        val = self.value # "823407561"
        val = val[:i] + val[i+size] + val[i+1:i+size] + val[i] + val[i+size+1:] # "8234" + "6" + "75" + "0" + "61" => "8234675061"
        child = State_Puzzle(val, self)
        self.children.append(child)

      if i > size - 1: # 0의 위치가 맨 윗줄에 있지 않다면 (0이 위치하는 index 가 0/1/2가 아니라면)
        val = self.value
        val = val[:i-size] + val[i] + val[i-size+1:i] + val[i-size] + val[i+1:]
        child = State_Puzzle(val, self)
        self.children.append(child)


In [5]:
# Astar 알고리즘을 실제 돌리는 코드
class AStar_solver:
  def __init__(self, start, goal):
    self.path = []
    self.visitedQueue = [] # closed list
    self.priorityQueue = PriorityQueue() # 우선순위 Queue (비용이랑 node를 함께 넣는 개념), open list에 해당
    self.start = start
    self.goal = goal

  def Solve(self):
    # parent는 해당 state를 방문하기 전에 어떤 state를 거쳐왔는지를 저장하는 List
    # start는 시작지점 state, goal은 목표 지점 state
    #                       ((현재상태), parent, start,     goal)
    startState = State_Puzzle(self.start, [], self.start, self.goal)
    # count 는 A*에 직접적으로 이용되지는 않을 것
    # 코드를 돌리는 이용자 입장에서, A* 알고리즘을 통해 총 몇 번의 탐색 과정을 거쳤는지 확인하기 위함
    count = 0 # 아무것도 세지 않은 상태

    # priorityQueue 는 fringe의 개념, 탐색 후보군들을 모아 놓는 list
    # (해당 후보군에 대한 cost, 해당 후보군을 몇번째로 탐색했는가, 탐색 후보군 state)
    # 여기서 해당 후보군에 대한 cost가 낮은 순서대로 priorityQueue에서 꺼내 탐색할 예정
    self.priorityQueue.put((0, count, startState)) # 제일먼저, start state를 집어넣는다. start state 에 대한 cost = 0

    # 앞서 정의해둔 cost에 따라, 비용을 최소화하는 방향으로 탐색을 진행하겠다.
    # self.priorityQueue는 open list와 유사한 개념
    # self.visitedQueue는 closed list와 유사한 개념
    # while loop 안에서는 priorityQueue(open list)에 있는 state를 한 개 한 개씩 꺼내서 goal state인지 확인하고, 아니라면 다음 state(children)를 확인하는 과정이 진행됩니다.
    # 1. priorityQueue에서 하나의 state를 뺀다.
    # 2. 해당 state 가 goal 인지 확인하고, 아니라면 해당 state의 다음 state를 priorityQueue에 추가한다.
    # 3. 이 과정을 self.path가 없거나, priorityQueue가 비어있을 때까지 반복합니다.
    # priorityQueue가 비어있을 때까지 반복이라 함은, startstate에서부터 탐색을 시작했을 때, 도달할 수 있는 모든 경로를 고려한다 라는 의미로 생각하실 수 있고
    # self.path가 비어있을 때까지 반복이라 함은 goal을 찾을 때까지 반복한다는 의미
    while (not self.path and self.priorityQueue.qsize()):
      closestChild = self.priorityQueue.get() [2] # cost가 가장 낮은 state를 뽑습니다.
      closestChild.CreateChildren() # 뽑은 대상에 대해서 다음 탐색 후보군을 탐색합니다.
      self.visitedQueue.append(closestChild.value) # 뽑은 state는 visitedQueue(closed list) 에 넣습니다.
      for child in closestChild.children: # 뽑은 대상에 대한 다음 탐색 후보군들, 이들에 대한 cost 산출
        if child.value not in self.visitedQueue: # 현재 탐색하고 있는 state가 이전에 탐색해본 노드가 아닌 경우에만 들어간다
          count += 1 # 몇번째로 해당 state를 탐색하는지
          if not child.dist: # child.dist=0인 경우, 이는 곧 goal을 찾은 경우를 의미함
            self.path = child.path # child에는 child로 오기까지 어떤 state를 거쳐 왔는지에 대한 정보가 path에 저장되어 있음
            break # goal을 찾았으니 더이상의 loop를 돌리지 않고 break
          self.priorityQueue.put((child.dist, count, child)) # goal을 찾지 못한 경우에는 cost, count, child를 openlist에 추가하여 다음 탐색 단계에서 탐색할 수 있따

    if not self.path:
      print("Goal of " + self.goal + " is not possible")

    return self.path

In [6]:
start = ("801526347")
goal = ("012345678")
print('starting...')

a = AStar_solver(start, goal)
a.Solve()

for i in range(len(a.path)):
  print(i)
  print(a.path[i][:3])
  print(a.path[i][3:6])
  print(a.path[i][6:9])
  print('\n')

starting...
0
801
526
347


1
081
526
347


2
581
026
347


3
581
206
347


4
581
260
347


5
580
261
347


6
508
261
347


7
058
261
347


8
258
061
347


9
258
361
047


10
258
361
407


11
258
301
467


12
258
310
467


13
250
318
467


14
205
318
467


15
025
318
467


16
325
018
467


17
325
418
067


18
325
418
607


19
325
418
670


20
325
410
678


21
320
415
678


22
302
415
678


23
312
405
678


24
312
045
678


25
012
345
678


